## X. Datenplausibilität & Umgang mit Ausreißern

In diesem Abschnitt werden die Daten des Wine-Quality-Datensatzes auf Plausibilität geprüft:

- Werte außerhalb fachlich definierter **weicher** Grenzen (Ausreißer, aber plausibel).
- Werte außerhalb **harter** Grenzen (physikalisch / fachlich nicht plausibel → potenzielle Datenfehler).

Die weichen/harten Grenzen sind so gewählt, dass sie typische Weinbereiche (pH, Alkohol, Restzucker, SO₂ etc.) abbilden, aber noch genügend Spielraum für Spezialfälle (z. B. süße Weine, stark geschwefelt) lassen.


In [8]:
# Nur für den temporären Import von df
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.3f}'.format


def load_wine_dataframe() -> pd.DataFrame:
    try:
        from ucimlrepo import fetch_ucirepo
    except ImportError:
        raise ImportError('ucimlrepo ist nicht installiert. Entweder installieren oder USE_LOCAL_CSV=True setzen.')
    wine = fetch_ucirepo(id=186)
    df_local = wine.data.original.copy()
    df_local.columns = (
        df_local.columns
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
    )
    return df_local
df = load_wine_dataframe()

[i] Lade Datensatz über ucimlrepo (id=186, Wine Quality)...
[i] Spalten: ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'ph', 'sulphates', 'alcohol', 'quality', 'color']
[i] Shape: (6497, 13)


### X.1 Berechnung von harten und weichen Grenzüberschreitungen

Alle harten Grenzwerte wurden aus wissenschaftlichen Arbeiten heraus übernommen.
Untere Grenzen (z. B. 0) wurden aus physikalischen Überlegungen abgeleitet, etwa dass Konzentrationen nicht negativ sein können.
Weiche Grenzen wurden ebenfalls übernommen, sofern angegeben, andernfalls sind diese nach eigenem Ermessen gewählt.

In [9]:
import pandas as pd
import numpy as np

data = df.copy()

numeric_cols = data.select_dtypes(include="number").columns.tolist()

# Fachlich begründetes Plausibilitäts-Schema (Soft/Hard-Bounds)
plausibility_schema = {
    "fixed acidity": {
        "soft_min": 4.0, "soft_max": 15.0,
        "hard_min": 3.0, "hard_max": 18.0,
    },
    "volatile acidity": {
        "soft_min": 0.1, "soft_max": 1.5,
        "hard_min": 0.0, "hard_max": 2.5,
    },
    "citric acid": {
        "soft_min": 0.0, "soft_max": 1.5,
        "hard_min": 0.0, "hard_max": 2.5,
    },
    "residual sugar": {
        "soft_min": 0.5, "soft_max": 40.0,
        "hard_min": 0.0, "hard_max": 80.0,
    },
    "chlorides": {
        "soft_min": 0.01, "soft_max": 0.5,
        "hard_min": 0.0, "hard_max": 1.0,
    },
    "free sulfur dioxide": {
        "soft_min": 1.0, "soft_max": 80.0,
        "hard_min": 0.0, "hard_max": 120.0,
    },
    "total sulfur dioxide": {
        "soft_min": 6.0, "soft_max": 300.0,
        "hard_min": 0.0, "hard_max": 400.0,
    },
    "density": {
        "soft_min": 0.985, "soft_max": 1.010,
        "hard_min": 0.980, "hard_max": 1.040,
    },
    "pH": {
        "soft_min": 2.6, "soft_max": 4.2,
        "hard_min": 2.2, "hard_max": 4.5,
    },
    "sulphates": {
        "soft_min": 0.2, "soft_max": 2.0,
        "hard_min": 0.0, "hard_max": 3.0,
    },
    "alcohol": {
        "soft_min": 8.0, "soft_max": 15.5,
        "hard_min": 5.0, "hard_max": 20.0,
    },
    # quality hat hier bewusst keine Grenzen, weil
    # es sich um ein Label/Rating handelt.
}

summary_rows = []
error_mask = pd.Series(False, index=data.index)  # harte Fehler (Hard-Bounds verletzt)
warn_mask = pd.Series(False, index=data.index)   # weiche Ausreißer (Soft-Bounds verletzt)

for col in numeric_cols:
    col_min = data[col].min()
    col_max = data[col].max()
    sch = plausibility_schema.get(col, {})
    soft_min = sch.get("soft_min", np.nan)
    soft_max = sch.get("soft_max", np.nan)
    hard_min = sch.get("hard_min", np.nan)
    hard_max = sch.get("hard_max", np.nan)

    col_error = pd.Series(False, index=data.index)
    col_warn = pd.Series(False, index=data.index)

    # Harte Grenzen → echte Plausibilitätsverletzungen
    if not np.isnan(hard_min):
        col_error |= data[col] < hard_min
    if not np.isnan(hard_max):
        col_error |= data[col] > hard_max

    # Weiche Grenzen → Ausreißer, aber potentiell plausible Spezialfälle
    if not np.isnan(soft_min):
        col_warn |= data[col] < soft_min
    if not np.isnan(soft_max):
        col_warn |= data[col] > soft_max

    # Warnungen nur dort, wo kein harter Fehler vorliegt
    col_warn &= ~col_error

    error_mask |= col_error
    warn_mask |= col_warn

    summary_rows.append({
        "column": col,
        "min": col_min,
        "max": col_max,
        "soft_min": soft_min,
        "soft_max": soft_max,
        "hard_min": hard_min,
        "hard_max": hard_max,
        "n_warn": int(col_warn.sum()),
        "n_error": int(col_error.sum()),
    })

plaus_summary = (
    pd.DataFrame(summary_rows)
    .set_index("column")
    .sort_index()
)

n_total = len(data)

# Anzahl Zeilen mit mindestens einer harten/weichen Verletzung
n_errors = int(error_mask.sum())
n_warns = int(warn_mask.sum())

# Zeilen mit tatsächlich verletzten Grenzen
hard_outliers = data[error_mask]
soft_outliers = data[warn_mask & ~error_mask]

display(plaus_summary)

print("\n--- Globale Übersicht ---")
print(f"Beobachtungen gesamt:            {n_total}")
print(f"Zeilen mit HARTem Fehler:        {n_errors}")
print(f"Zeilen mit weicher Warnung:      {n_warns}")

print("\nBeispiele weicher Ausreißer (head):")
display(soft_outliers.head())

print("\nBeispiele harter Ausreißer (head):")
display(hard_outliers.head())


,min,max,soft_min,soft_max,hard_min,hard_max,n_warn,n_error
column,,,,,,,,
alcohol,8.000,14.900,8.000,15.500,5.000,20.000,0,0
chlorides,0.009,0.611,0.010,0.500,0.000,1.000,3,0
citric_acid,0.000,1.660,NaN,NaN,NaN,NaN,0,0
density,0.987,1.039,0.985,1.010,0.980,1.040,3,0
fixed_acidity,3.800,15.900,NaN,NaN,NaN,NaN,0,0
free_sulfur_dioxide,1.000,289.000,NaN,NaN,NaN,NaN,0,0
ph,2.720,4.010,NaN,NaN,NaN,NaN,0,0
quality,3.000,9.000,NaN,NaN,NaN,NaN,0,0
residual_sugar,0.600,65.800,NaN,NaN,NaN,NaN,0,0



--- Globale Übersicht ---
Beobachtungen gesamt:            6497
Zeilen mit HARTem Fehler:        0
Zeilen mit weicher Warnung:      6

Beispiele weicher Ausreißer (head):


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality,color
151,9.200,0.520,1.000,3.400,0.610,32.000,69.000,1.000,2.740,2.000,9.400,4,red
258,7.700,0.410,0.760,1.800,0.611,8.000,45.000,0.997,3.060,1.260,9.400,5,red
3252,7.900,0.330,0.280,31.600,0.053,35.000,176.000,1.010,3.150,0.380,8.800,6,white
3262,7.900,0.330,0.280,31.600,0.053,35.000,176.000,1.010,3.150,0.380,8.800,6,white
4380,7.800,0.965,0.600,65.800,0.074,8.000,160.000,1.039,3.390,0.690,11.700,6,white



Beispiele harter Ausreißer (head):


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality,color


### X.2 Interpretation der Plausibilitätsanalyse

**Harte Grenzen (Fehler)**

- Die Auswertung zeigt, dass **keine Zeilen mit harten Plausibilitätsverletzungen** existieren.
- Das bedeutet: alle Messwerte liegen innerhalb der fachlich möglichen Bereiche für Wein (pH, Alkoholgehalt, Restzucker, SO₂ etc.).

**Weiche Grenzen (Ausreißer)**

- Einige Beobachtungen liegen außerhalb der typischen „Alltagsbereiche“ (z. B. sehr hohe Restzuckerwerte bei süßen Weinen, hohe Gesamt-SO₂-Werte oder auffällig hoher Säuregehalt).
- Diese Werte sind jedoch aus önologischer Sicht plausibel (z. B. lieblich/süß ausgebaute Weine, stark geschwefelt, sehr säurebetont).
- Diese werden daher als **reale Extremfälle** und nicht als Messfehler.

**Entscheidung: Sollen Datensätze entfernt werden?**

- **Nein, es werden keine Zeilen aufgrund der Plausibilitätsprüfung entfernt.**
- Begründung:
  - Es gibt **keine klar unmöglichen Werte** (z. B. negative Konzentrationen, Alkohol > 20 Vol.-% oder pH > 7).
  - Die als „Warnung“ markierten Werte sind **fachlich erklärbare Ausreißer** (z. B. süßere Weine, stärker geschwefelt, spezielle Stilistik).
  - Das Entfernen dieser Ausreißer würde die reale Varianz des Datensatzes künstlich verringern und könnte das Modell systematisch verzerren (Bias).
